# ELO MERCHANT CATEGORY RECOMMENDATION 
### COMPETITION ON KAGGLE
## Data Transformations of new_merchant_transactions.csv
### 1-1-2019

In [1]:
## Variables specific for competition

ID = 'card_id'                                            
TARGET = 'target'    

RAW_DIRECTORY = 'C:/Users/judit/Documents/learning/kaggle/Elo_201812/rawdata/'  
DIRECTORY = 'C:/Users/judit/Documents/learning/kaggle/Elo_201812/data/'
HIST_TRANS_FILE = RAW_DIRECTORY + 'historical_transactions.csv'
MERCHANTS_FILE = RAW_DIRECTORY + 'merchants.csv'
NEW_MERCH_TRANS_FILE = RAW_DIRECTORY + 'new_merchant_transactions.csv'
TRAIN_FILE = RAW_DIRECTORY + 'train.csv'    
TEST_FILE = RAW_DIRECTORY +'test.csv'
SAMPLE_SUBMISSION_FILE = RAW_DIRECTORY + 'sample_submission.csv'

REF_DATES_FILE = DIRECTORY + 'ref_dates.pkl'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
#import missingno
import pickle

random.seed(1)

In [3]:
def reduce_mem_usage(df, verbose = True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem\
                                                                                                   ) / start_mem))
    return df

def create_date_features(df, source_column = 'purchase_date', preposition_of_new = 'purchase'):
    df[preposition_of_new + '_year'] = df[source_column].dt.year
    df[preposition_of_new + '_month'] = df[source_column].dt.month
    df[preposition_of_new + '_day'] = df[source_column].dt.day
    df[preposition_of_new + '_hour'] = df[source_column].dt.hour
    df[preposition_of_new + '_weekofyear'] = df[source_column].dt.weekofyear
    df[preposition_of_new + '_dayofweek'] = df[source_column].dt.dayofweek
    df[preposition_of_new + '_quarter'] = df[source_column].dt.quarter
    
    return df

In [4]:
hist = pd.read_csv(NEW_MERCH_TRANS_FILE, parse_dates=["purchase_date"])
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [5]:
hist.shape

(1963031, 14)

In [6]:
hist['authorized_flag'].unique()

array(['Y'], dtype=object)

In [7]:
hist[hist['authorized_flag'].isnull()].shape

(0, 14)

The new_merchants_transactions.csv includes only authorized transactions!!!

In [8]:
hist['authorized_flag'] = hist['authorized_flag'].apply(lambda x : 1 if x == 'Y' else 0)

In [9]:
hist[hist['card_id'].isnull()].shape

(0, 14)

In [10]:
# card_id is a hexadecimal number. Convert it into decimal.
hist['card_id'] = hist['card_id'].apply(lambda s : int(s[5:], 16))
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,280711374089,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,1,280711374089,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,1,280711374089,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,1,280711374089,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,1,1027936849227,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [11]:
hist[hist['city_id'].isnull()].shape, hist['city_id'].min(), hist['city_id'].max()

((0, 14), -1, 347)

In [12]:
hist['city_id'] = hist['city_id'].apply(lambda x : np.nan if x == -1 else x)

In [13]:
hist['category_1'].unique()

array(['N', 'Y'], dtype=object)

In [14]:
hist[hist['category_1'] == 'Y'].shape, hist[hist['category_1'] == 'N'].shape

((63096, 14), (1899935, 14))

In [15]:
hist[hist['category_1'].isnull()].shape

(0, 14)

In [16]:
hist['category_1'] = hist['category_1'].apply(lambda x : 1 if x == 'Y' else 0)

In [17]:
hist[hist['installments'].isnull()].shape

(0, 14)

In [18]:
hist['installments'].describe()

count    1.963031e+06
mean     6.829643e-01
std      1.584069e+00
min     -1.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      9.990000e+02
Name: installments, dtype: float64

In [19]:
hist.loc[hist['installments'] < 999, 'installments'].describe()

count    1.963029e+06
mean     6.819471e-01
std      1.222239e+00
min     -1.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.200000e+01
Name: installments, dtype: float64

In [20]:
hist[hist['installments'] == 999].shape

(2, 14)

In [21]:
hist[hist['installments'] == -1].shape

(55922, 14)

In [22]:
hist['category_3'].unique()

array(['B', nan, 'C', 'A'], dtype=object)

In [23]:
hist[hist['category_3'].isnull()].shape

(55922, 14)

In [24]:
hist.loc[hist['category_3'].isnull(), 'installments'].unique()

array([-1], dtype=int64)

In [25]:
hist.loc[hist['category_3'] == 'A', 'installments'].unique()

array([0], dtype=int64)

In [26]:
hist.loc[hist['category_3'] == 'B', 'installments'].unique()

array([1], dtype=int64)

In [27]:
hist.loc[hist['category_3'] == 'C', 'installments'].unique()

array([  3,   2,  12,   6,   4,  10,   5,   9,   8,   7,  11, 999], dtype=int64)

In [28]:
hist[hist['category_3'] == 'A'].shape, hist[hist['category_3'] == 'B'].shape, hist[hist['category_3'] == 'C'].shape

((922244, 14), (836178, 14), (148687, 14))

In [29]:
hist.loc[hist['category_3'] == 'A', 'card_id'].nunique(), hist.loc[hist['category_3'] == 'B', 'card_id'].nunique(),\
hist.loc[hist['category_3'] == 'C', 'card_id'].nunique(), hist.loc[hist['category_3'].isnull(), 'card_id'].nunique()

(140197, 139142, 81261, 40708)

In [30]:
hist['card_id'].nunique()

290001

In [31]:
hist.loc[hist['category_3'] == 'A', 'card_id'].nunique() + hist.loc[hist['category_3'] == 'B', 'card_id'].nunique() +\
hist.loc[hist['category_3'] == 'C', 'card_id'].nunique() + hist.loc[hist['category_3'].isnull(), 'card_id'].nunique()

401308

For the same card_id, different values of category_3 can be found.

In [32]:
# Valeria Elo said that both 999 and -1 refer to missing values 
# (https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/72993)
hist['installments'] = hist['installments'].apply(lambda x : np.nan if x == -1 else np.nan if x == 999 else x)
hist[hist['installments'].isnull()].shape, hist[hist['installments'] == -1].shape, hist[hist['installments'] == 999].shape

((55924, 14), (0, 14), (0, 14))

In [33]:
hist['category_3'] = hist['category_3'].apply(lambda s : 0 if s == 'A' else 1 if s == 'B' else 2 if s == 'C' else np.nan)
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,280711374089,107.0,0,1.0,1.0,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,1,280711374089,140.0,0,1.0,1.0,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,1,280711374089,330.0,0,1.0,1.0,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,1,280711374089,NaN,1,1.0,1.0,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,1,1027936849227,NaN,1,1.0,1.0,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [34]:
hist[hist['merchant_category_id'].isnull()].shape

(0, 14)

In [35]:
hist['merchant_category_id'].nunique(), hist['merchant_category_id'].min(), hist['merchant_category_id'].max()

(314, -1, 891)

In [36]:
hist['merchant_category_id'] = hist['merchant_category_id'].apply(lambda x : np.nan if x == -1 else x)

In [37]:
hist[hist['merchant_id'].isnull()].shape

(26216, 14)

In [38]:
# merchant_id is a hexadecimal number. Convert it into decimal.
hist['merchant_id'] = hist['merchant_id'].apply(lambda s : int(s[5:], 16) if pd.notnull(s) else np.nan)
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,280711374089,107.0,0,1.0,1.0,307.0,7.592625e+11,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,1,280711374089,140.0,0,1.0,1.0,307.0,5.865658e+11,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,1,280711374089,330.0,0,1.0,1.0,507.0,7.444087e+11,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,1,280711374089,NaN,1,1.0,1.0,661.0,6.808329e+11,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,1,1027936849227,NaN,1,1.0,1.0,166.0,2.599626e+11,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [39]:
hist['merchant_id'].describe()

count    1.936815e+06
mean     5.503235e+11
std      3.188294e+11
min      2.429567e+06
25%      2.785781e+11
50%      5.553498e+11
75%      8.277068e+11
max      1.099496e+12
Name: merchant_id, dtype: float64

In [40]:
sorted(hist['month_lag'].unique())

[1, 2]

In [41]:
hist[hist['month_lag'].isnull()].shape

(0, 14)

In [42]:
hist['purchase_amount'].describe()

count    1.963031e+06
mean    -5.509690e-01
std      6.940043e-01
min     -7.468928e-01
25%     -7.166294e-01
50%     -6.748406e-01
75%     -5.816162e-01
max      2.631575e+02
Name: purchase_amount, dtype: float64

In [43]:
hist[hist['purchase_amount'].isnull()].shape

(0, 14)

In [44]:
hist['purchase_amount'].quantile([.0001, .001, .01, .1, .9, .99, .999, .9999, .99999, .999999, .9999999, .99999995, .99999999])

0.000100     -0.746893
0.001000     -0.746156
0.010000     -0.742400
0.100000     -0.731881
0.900000     -0.371395
0.990000      1.462817
0.999000      6.775544
0.999900     21.792860
0.999990     54.970249
0.999999     98.563941
1.000000    238.795559
1.000000    250.976529
1.000000    260.721304
Name: purchase_amount, dtype: float64

In [45]:
hist['purchase_date'].describe()

count                 1963031
unique                1667025
top       2017-11-18 00:00:00
freq                      582
first     2017-03-01 03:24:51
last      2018-04-30 23:59:59
Name: purchase_date, dtype: object

In [46]:
hist[hist['purchase_date'].isnull()].shape

(0, 14)

In [47]:
hist = create_date_features(df = hist, source_column = 'purchase_date', preposition_of_new = 'purchase')

hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,category_2,state_id,subsector_id,purchase_year,purchase_month,purchase_day,purchase_hour,purchase_weekofyear,purchase_dayofweek,purchase_quarter
0,1,280711374089,107.0,0,1.0,1.0,307.0,7.592625e+11,1,-0.557574,...,1.0,9,19,2018,3,11,14,10,6,1
1,1,280711374089,140.0,0,1.0,1.0,307.0,5.865658e+11,1,-0.569580,...,1.0,9,19,2018,3,19,18,12,0,1
2,1,280711374089,330.0,0,1.0,1.0,507.0,7.444087e+11,2,-0.551037,...,1.0,9,14,2018,4,26,14,17,3,2
3,1,280711374089,NaN,1,1.0,1.0,661.0,6.808329e+11,1,-0.671925,...,NaN,-1,8,2018,3,7,9,10,2,1
4,1,1027936849227,NaN,1,1.0,1.0,166.0,2.599626e+11,1,-0.659904,...,NaN,-1,29,2018,3,22,21,12,3,1


In [48]:
hist['category_2'].unique()

array([  1.,  nan,   3.,   2.,   4.,   5.])

In [49]:
print(set(hist.loc[hist['category_2'] == 1,'state_id']))             # either Central-West or Southeast region
print(set(hist.loc[hist['category_2'] == 2,'state_id']))             # North region
print(set(hist.loc[hist['category_2'] == 3,'state_id']))             # Northeast region
print(set(hist.loc[hist['category_2'] == 4,'state_id']))             # either Central-West of Southeast region
print(set(hist.loc[hist['category_2'] == 5,'state_id']))             # South region
print(set(hist.loc[hist['category_2'].isnull(),'state_id']))         # unknown/abroad

{16, 9, 12, 15}
{6, 10, 18, 23, 24}
{1, 2, 3, 7, 8, 11, 17, 19}
{4, 13, 22, 14}
{5, 20, 21}
{-1}


In [50]:
hist.loc[hist['category_1'] == 1, 'category_2'].unique()

array([ nan])

In [51]:
hist.loc[hist['category_1'] == 1, 'city_id'].unique()

array([ nan])

In [52]:
hist.loc[hist['category_1'] == 1, 'state_id'].unique()                     

array([-1], dtype=int64)

In [53]:
hist.loc[hist['category_2'].isnull(), 'category_1'].unique()

array([1, 0], dtype=int64)

In [54]:
# category_2 refers to the 5 regions of Brazil. There is also a federal district. NaN states both for the federal district 
# and for foreign purchases. Let's impute 6 for the federal district
hist['category_2'] = hist.apply(lambda row : row['category_2'] if (pd.notnull(row['category_2'])) else 
                                             6 if row['category_3'] == 0 else np.nan, axis = 1)

hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,category_2,state_id,subsector_id,purchase_year,purchase_month,purchase_day,purchase_hour,purchase_weekofyear,purchase_dayofweek,purchase_quarter
0,1,280711374089,107.0,0,1.0,1.0,307.0,7.592625e+11,1,-0.557574,...,1.0,9,19,2018,3,11,14,10,6,1
1,1,280711374089,140.0,0,1.0,1.0,307.0,5.865658e+11,1,-0.569580,...,1.0,9,19,2018,3,19,18,12,0,1
2,1,280711374089,330.0,0,1.0,1.0,507.0,7.444087e+11,2,-0.551037,...,1.0,9,14,2018,4,26,14,17,3,2
3,1,280711374089,NaN,1,1.0,1.0,661.0,6.808329e+11,1,-0.671925,...,NaN,-1,8,2018,3,7,9,10,2,1
4,1,1027936849227,NaN,1,1.0,1.0,166.0,2.599626e+11,1,-0.659904,...,NaN,-1,29,2018,3,22,21,12,3,1


In [55]:
hist[hist['state_id'].isnull()].shape, hist['state_id'].min()

((0, 21), -1)

In [56]:
hist['state_id'] = hist['state_id'].apply(lambda x : np.nan if x == -1 else x)

In [57]:
hist[hist['subsector_id'].isnull()].shape, hist['subsector_id'].min()

((0, 21), -1)

In [58]:
hist['subsector_id'] = hist['subsector_id'].apply(lambda x : np.nan if x == -1 else x)

In [59]:
hist = reduce_mem_usage(hist, verbose = True)

Starting memory usage: 314.51 MB
Reduced memory usage: 87.99 MB (72.0% reduction)


### Aggregates

In [60]:
def aggregate_features(df = hist, prefix = '', suffix = '', gb_cols = ['card_id'], 
                       agg_func = {'purchase_amount': ['sum', 'mean', 'max', 'min', 'std', 'size'],
                                   'installments': ['sum', 'median', 'mean', 'max', 'min', 'std']},
                       authorized = True, non_authorized = True, domestic = True, non_domestic = True,
                       max_month_lag = None, max_week_lag = None,
                       min_installments = None, max_installments = None): 
    if not authorized:
        df = df[df['authorized_flag'] == 0]
    if not non_authorized:
        df = df[df['authorized_flag'] == 1]
    if not domestic:
        df = df[df['category_2'].isnull()]
    if not non_domestic:
        df = df[df['category_2'].notnull()]
    if max_month_lag:
        df = df[df['month_lag'] <= max_month_lag]
    if max_week_lag:
        df = df[df['week_lag'] <= max_week_lag]
    if min_installments:
        df = df[df['installments'] >= min_installments]
    if max_installments:
        df = df[df['installments'] <= max_installments]
    agg_df = df.groupby(gb_cols).agg(agg_func)
    agg_df.columns = [prefix + '_'.join(col).strip() + suffix for col in agg_df.columns.values]  
    agg_df.reset_index(inplace = True)

    return agg_df

In [61]:
# Load reference dates defined in 8_Datatransf_historical_transactions_190101.ipynb
ref_dates = pd.read_pickle(REF_DATES_FILE)
ref_dates.head()

,card_id,reference_date_min
0,7377857,2018-02-01 05:06:36
1,19103846,2018-02-01 19:42:58
2,22048496,2018-02-01 07:20:40
3,24721286,2017-10-01 02:08:20
4,25427418,2018-02-01 08:03:55


In [62]:
ref_dates.columns = ['card_id', 'reference_date']

In [63]:
hist = hist.merge(ref_dates, how = 'left', on = 'card_id')
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,state_id,subsector_id,purchase_year,purchase_month,purchase_day,purchase_hour,purchase_weekofyear,purchase_dayofweek,purchase_quarter,reference_date
0,1,280711374089,107.0,0,1.0,1.0,307.0,7.592625e+11,1,-0.557617,...,9.0,19.0,2018,3,11,14,10,6,1,2018-02-04 21:08:12
1,1,280711374089,140.0,0,1.0,1.0,307.0,5.865659e+11,1,-0.569336,...,9.0,19.0,2018,3,19,18,12,0,1,2018-02-04 21:08:12
2,1,280711374089,330.0,0,1.0,1.0,507.0,7.444087e+11,2,-0.551270,...,9.0,14.0,2018,4,26,14,17,3,2,2018-02-04 21:08:12
3,1,280711374089,NaN,1,1.0,1.0,661.0,6.808329e+11,1,-0.671875,...,NaN,8.0,2018,3,7,9,10,2,1,2018-02-04 21:08:12
4,1,1027936849227,NaN,1,1.0,1.0,166.0,2.599626e+11,1,-0.659668,...,NaN,29.0,2018,3,22,21,12,3,1,2018-02-01 02:22:38


In [64]:
###########################################################################
## Check the harmony between reference dates, purchase dates and month_lags
###########################################################################

hist['calculated_lag'] = (hist['purchase_date'] - hist['reference_date']) / np.timedelta64(1, 'M')

print(hist[hist['calculated_lag'] != hist['calculated_lag']].shape)

hist[hist['month_lag'] != hist['calculated_lag']].head()

(0, 23)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,subsector_id,purchase_year,purchase_month,purchase_day,purchase_hour,purchase_weekofyear,purchase_dayofweek,purchase_quarter,reference_date,calculated_lag
0,1,280711374089,107.0,0,1.0,1.0,307.0,7.592625e+11,1,-0.557617,...,19.0,2018,3,11,14,10,6,1,2018-02-04 21:08:12,1.141465
1,1,280711374089,140.0,0,1.0,1.0,307.0,5.865659e+11,1,-0.569336,...,19.0,2018,3,19,18,12,0,1,2018-02-04 21:08:12,1.409689
2,1,280711374089,330.0,0,1.0,1.0,507.0,7.444087e+11,2,-0.551270,...,14.0,2018,4,26,14,17,3,2,2018-02-04 21:08:12,2.651675
3,1,280711374089,NaN,1,1.0,1.0,661.0,6.808329e+11,1,-0.671875,...,8.0,2018,3,7,9,10,2,1,2018-02-04 21:08:12,1.002876
4,1,1027936849227,NaN,1,1.0,1.0,166.0,2.599626e+11,1,-0.659668,...,29.0,2018,3,22,21,12,3,1,2018-02-01 02:22:38,1.635563


In [65]:
hist['lagdiff'] = hist['calculated_lag'] - hist['month_lag']

print(hist[((hist['lagdiff'] >= 1) | (hist['lagdiff'] <= -1))].shape)

hist[((hist['lagdiff'] >= 1) | (hist['lagdiff'] <= -1))].head()

(2074, 24)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_year,purchase_month,purchase_day,purchase_hour,purchase_weekofyear,purchase_dayofweek,purchase_quarter,reference_date,calculated_lag,lagdiff
950,1,200001111054,137.0,0,0.0,0.0,80.0,2.986535e+11,2,-0.681641,...,2018,1,31,23,5,2,1,2017-11-01 00:12:26,3.022216,1.022216
951,1,200001111054,137.0,0,0.0,0.0,307.0,9.259335e+11,2,-0.671875,...,2018,1,31,23,5,2,1,2017-11-01 00:12:26,3.021937,1.021937
1015,1,1075540040452,NaN,1,4.0,2.0,210.0,4.311888e+11,1,8.468750,...,2018,1,31,15,5,2,1,2017-12-01 00:00:00,2.025797,1.025797
2014,1,865564038120,253.0,0,0.0,0.0,367.0,1.990933e+11,1,-0.710938,...,2017,8,31,14,35,3,3,2017-07-01 10:23:54,2.010084,1.010084
5057,1,78305994616,69.0,0,0.0,0.0,561.0,3.477155e+11,2,-0.671875,...,2017,10,31,22,44,1,4,2017-08-01 05:50:27,3.012195,1.012195


In [66]:
hist['lagdiff'].describe()

count    1.963031e+06
mean     4.104037e-01
std      2.972678e-01
min     -9.664736e-01
25%      1.586001e-01
50%      4.053988e-01
75%      6.616510e-01
max      1.036750e+00
Name: lagdiff, dtype: float64

In [67]:
hist['calculated_lag'].describe()

count    1.963031e+06
mean     1.886919e+00
std      5.843209e-01
min      3.352643e-02
25%      1.382722e+00
50%      1.874768e+00
75%      2.379086e+00
max      3.022365e+00
Name: calculated_lag, dtype: float64

In [68]:
lag_agg = aggregate_features(df = hist, prefix = '', suffix = '', gb_cols = ['card_id'], 
                             agg_func = {'calculated_lag': ['max', 'min'],
                                         'month_lag': ['max', 'min']},
                             authorized = True, non_authorized = True, domestic = True, non_domestic = True,
                             max_month_lag = None, max_week_lag = None,
                             min_installments = None, max_installments = None)
lag_agg.head()

,card_id,calculated_lag_max,calculated_lag_min,month_lag_max,month_lag_min
0,7377857,2.216732,2.012521,2,2
1,19103846,2.891561,0.915955,2,1
2,22048496,1.612487,1.433324,1,1
3,24721286,3.010958,1.497088,2,1
4,25427418,2.900720,0.958781,2,1


In [69]:
lag_agg['calculated_lag_range'] = lag_agg['calculated_lag_max'] - lag_agg['calculated_lag_min']
lag_agg['month_lag_range'] = lag_agg['month_lag_max'] - lag_agg['month_lag_min']
lag_agg['lag_range_diff'] = lag_agg['calculated_lag_range'] - lag_agg['month_lag_range']
lag_agg['lag_range_diff'].describe()

count    290001.000000
mean          0.310410
std           0.365963
min          -0.999598
25%           0.000000
50%           0.277715
75%           0.617816
max           1.034415
Name: lag_range_diff, dtype: float64

As the difference between the lag ranges is sometimes larger than 1, it seems not the same reference date is used in this file and in historical_transactions.csv.  
So far we didn't use the reference date, and the differences are between -1 and 1.03 with mean at 0.3, so I think that we don't neccessarily need to deal with these differences.

In [70]:
del lag_agg
hist.drop(['calculated_lag', 'lagdiff'], inplace = True, axis = 1)

In [71]:
###########################################################################
## End of checking month_lag
###########################################################################

In [72]:
hist = create_date_features(hist, source_column = 'reference_date', preposition_of_new = 'reference')
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_dayofweek,purchase_quarter,reference_date,reference_year,reference_month,reference_day,reference_hour,reference_weekofyear,reference_dayofweek,reference_quarter
0,1,280711374089,107.0,0,1.0,1.0,307.0,7.592625e+11,1,-0.557617,...,6,1,2018-02-04 21:08:12,2018,2,4,21,5,6,1
1,1,280711374089,140.0,0,1.0,1.0,307.0,5.865659e+11,1,-0.569336,...,0,1,2018-02-04 21:08:12,2018,2,4,21,5,6,1
2,1,280711374089,330.0,0,1.0,1.0,507.0,7.444087e+11,2,-0.551270,...,3,2,2018-02-04 21:08:12,2018,2,4,21,5,6,1
3,1,280711374089,NaN,1,1.0,1.0,661.0,6.808329e+11,1,-0.671875,...,2,1,2018-02-04 21:08:12,2018,2,4,21,5,6,1
4,1,1027936849227,NaN,1,1.0,1.0,166.0,2.599626e+11,1,-0.659668,...,3,1,2018-02-01 02:22:38,2018,2,1,2,5,3,1


In [73]:
hist[['card_id', 'purchase_date', 'reference_date', 'month_lag']].head(10)

,card_id,purchase_date,reference_date,month_lag
0,280711374089,2018-03-11 14:57:36,2018-02-04 21:08:12,1
1,280711374089,2018-03-19 18:53:37,2018-02-04 21:08:12,1
2,280711374089,2018-04-26 14:08:44,2018-02-04 21:08:12,2
3,280711374089,2018-03-07 09:43:21,2018-02-04 21:08:12,1
4,1027936849227,2018-03-22 21:07:53,2018-02-01 02:22:38,1
5,1027936849227,2018-04-02 12:53:28,2018-02-01 02:22:38,2
6,1027936849227,2018-03-28 19:50:19,2018-02-01 02:22:38,1
7,1027936849227,2018-04-05 08:06:52,2018-02-01 02:22:38,2
8,1027936849227,2018-04-07 18:37:40,2018-02-01 02:22:38,2
9,1027936849227,2018-03-17 18:10:41,2018-02-01 02:22:38,1


In [74]:
hist['week_lag'] = (hist.purchase_date.dt.year - hist.reference_date.dt.year
                   ) * 52 + (hist.purchase_date.dt.weekofyear - hist.reference_date.dt.weekofyear)
hist[['card_id', 'purchase_date', 'reference_date', 'month_lag', 'week_lag']].head()

,card_id,purchase_date,reference_date,month_lag,week_lag
0,280711374089,2018-03-11 14:57:36,2018-02-04 21:08:12,1,5
1,280711374089,2018-03-19 18:53:37,2018-02-04 21:08:12,1,7
2,280711374089,2018-04-26 14:08:44,2018-02-04 21:08:12,2,12
3,280711374089,2018-03-07 09:43:21,2018-02-04 21:08:12,1,5
4,1027936849227,2018-03-22 21:07:53,2018-02-01 02:22:38,1,7


In [75]:
hist.to_pickle(DIRECTORY + 'new_trans.pkl')

In [76]:
hist.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'purchase_year', 'purchase_month', 'purchase_day',
       'purchase_hour', 'purchase_weekofyear', 'purchase_dayofweek',
       'purchase_quarter', 'reference_date', 'reference_year',
       'reference_month', 'reference_day', 'reference_hour',
       'reference_weekofyear', 'reference_dayofweek', 'reference_quarter',
       'week_lag'],
      dtype='object')

In [77]:
def mode(series):
    if len(series.mode()) > 0:
        return series.mode().iloc[0]
    else:
        return np.nan

def nancnt(series):
    '''Returns the count of NaN values'''
    return series.isnull().sum()

def nanperc(series):
    '''Returns the percentile of NaN values'''
    return 100 * series.isnull().sum() / len(series)

agg_func = {
    'city_id'              : ['nunique', mode, nancnt, nanperc],
    'category_1'           : ['sum', 'mean'],
    'installments'         : ['sum', 'median', 'mean', 'max', 'min', 'std', mode, nancnt, nanperc],
    'category_3'           : ['nunique', mode, nancnt, nanperc],
    'merchant_category_id' : ['nunique', mode, nancnt, nanperc],
    'merchant_id'          : ['nunique', mode, nancnt, nanperc],
    'month_lag'            : ['mean'],
    'purchase_amount'      : ['sum', 'median', 'mean', 'max', 'min', 'std', mode, 'size'],
    'category_2'           : ['nunique', mode, nancnt, nanperc],
    'state_id'             : ['nunique', mode, nancnt, nanperc],
    'subsector_id'         : ['nunique', mode, nancnt, nanperc],
    'purchase_year'        : ['mean', 'median', 'max', 'min', 'std', mode],
    'purchase_month'       : ['mean', 'median', 'max', 'min', 'std', mode],
    'purchase_day'         : ['mean', 'median', 'max', 'min', 'std', mode],
    'purchase_hour'        : ['mean', 'median', 'max', 'min', 'std', mode],
    'purchase_weekofyear'  : ['mean', 'median', 'max', 'min', 'std', mode],
    'purchase_dayofweek'   : ['mean', 'median', 'max', 'min', 'std', mode],
    'purchase_quarter'     : ['mean', 'median', 'max', 'min', 'std', mode],
    'week_lag'             : ['median', 'mean', 'max', 'min', 'std', mode]
}

# Aggregate features, use  
# - both domestic and non-domestic transactions,
# - for the whole evaluation period.

hist_agg_1 = aggregate_features(df = hist, prefix = 'new_', suffix = '_all', gb_cols = ['card_id'], agg_func = agg_func,
                                authorized = True, non_authorized = True, 
                                domestic = True, non_domestic = True,
                                max_month_lag = None, max_week_lag = None,
                                min_installments = None, max_installments = None)

In [78]:
hist_agg_1.head()

,card_id,new_city_id_nunique_all,new_city_id_mode_all,new_city_id_nancnt_all,new_city_id_nanperc_all,new_category_1_sum_all,new_category_1_mean_all,new_installments_sum_all,new_installments_median_all,new_installments_mean_all,...,new_purchase_quarter_max_all,new_purchase_quarter_min_all,new_purchase_quarter_std_all,new_purchase_quarter_mode_all,new_week_lag_median_all,new_week_lag_mean_all,new_week_lag_max_all,new_week_lag_min_all,new_week_lag_std_all,new_week_lag_mode_all
0,7377857,2,69.0,0.0,0.000000,0,0.000000,2.0,1.0,1.000000,...,2,2,0.000000,2,9.5,9.500000,10,9,0.707107,9
1,19103846,7,314.0,3.0,11.539062,2,0.076923,43.0,1.0,1.719727,...,2,1,0.485165,1,7.0,7.846154,13,4,2.633512,6
2,22048496,1,137.0,0.0,0.000000,0,0.000000,0.0,0.0,0.000000,...,1,1,0.000000,1,6.5,6.500000,7,6,0.707107,6
3,24721286,7,69.0,0.0,0.000000,0,0.000000,0.0,0.0,0.000000,...,4,4,0.000000,4,9.0,9.387097,13,7,2.108355,8
4,25427418,2,161.0,0.0,0.000000,0,0.000000,17.0,1.0,1.700195,...,2,1,0.467099,1,5.0,6.818182,13,4,3.250175,4


In [79]:
hist_agg_1.to_pickle(DIRECTORY + 'new_agg_1.pkl')

In [80]:
# Aggregate features, use 
# - both domestic and non-domestic transactions,
# - for the first 1 month of evaluation period.

hist_agg_2 = aggregate_features(df = hist, prefix = 'new_', suffix = '_lag1m', gb_cols = ['card_id'], agg_func = agg_func,
                                authorized = True, non_authorized = True, 
                                domestic = True, non_domestic = True,
                                max_month_lag = 1, max_week_lag = None,
                                min_installments = None, max_installments = None)

In [81]:
hist_agg_2.to_pickle(DIRECTORY + 'new_agg_2.pkl')
hist_agg_2.head()

,card_id,new_city_id_nunique_lag1m,new_city_id_mode_lag1m,new_city_id_nancnt_lag1m,new_city_id_nanperc_lag1m,new_category_1_sum_lag1m,new_category_1_mean_lag1m,new_installments_sum_lag1m,new_installments_median_lag1m,new_installments_mean_lag1m,...,new_purchase_quarter_max_lag1m,new_purchase_quarter_min_lag1m,new_purchase_quarter_std_lag1m,new_purchase_quarter_mode_lag1m,new_week_lag_median_lag1m,new_week_lag_mean_lag1m,new_week_lag_max_lag1m,new_week_lag_min_lag1m,new_week_lag_std_lag1m,new_week_lag_mode_lag1m
0,19103846,7,69.0,2.0,11.765625,1,0.058824,20.0,1.0,1.250,...,1,1,0.0,1,6.0,6.235294,8,4,1.347656,6
1,22048496,1,137.0,0.0,0.000000,0,0.000000,0.0,0.0,0.000,...,1,1,0.0,1,6.5,6.500000,7,6,0.707107,6
2,24721286,4,69.0,0.0,0.000000,0,0.000000,0.0,0.0,0.000,...,4,4,0.0,4,8.0,8.095238,9,7,0.768424,8
3,25427418,2,161.0,0.0,0.000000,0,0.000000,15.0,1.5,1.875,...,1,1,0.0,1,5.0,5.125000,8,4,1.356203,4
4,40934234,2,69.0,0.0,0.000000,0,0.000000,2.0,1.0,1.000,...,1,1,0.0,1,4.0,4.000000,4,4,0.000000,4


In [82]:
# Aggregate features, use 
# - only domestic transactions,
# - for the whole evaluation period.

hist_agg_3 = aggregate_features(df = hist, prefix = 'new_', suffix = '_dom_all', gb_cols = ['card_id'], agg_func = agg_func,
                                authorized = True, non_authorized = True, 
                                domestic = True, non_domestic = False,
                                max_month_lag = None, max_week_lag = None,
                                min_installments = None, max_installments = None)
hist_agg_3.to_pickle(DIRECTORY + 'new_agg_3.pkl')

In [83]:
# Aggregate features, use 
# - only domestic transactions,
# - for the first month of evaluation period.

hist_agg_4 = aggregate_features(df = hist, prefix = 'new_', suffix = '_dom_lag1', gb_cols = ['card_id'], agg_func = agg_func,
                                authorized = True, non_authorized = True, 
                                domestic = True, non_domestic = False,
                                max_month_lag = 1, max_week_lag = None,
                                min_installments = None, max_installments = None)
hist_agg_4.to_pickle(DIRECTORY + 'new_agg_4.pkl')

In [84]:
# Aggregate features, use 
# - only non-domestic transactions,
# - for the whole evaluation period.

hist_agg_5 = aggregate_features(df = hist, prefix = 'new_', suffix = '_nondom_all', gb_cols = ['card_id'], agg_func = agg_func,
                                authorized = True, non_authorized = True, 
                                domestic = False, non_domestic = True,
                                max_month_lag = None, max_week_lag = None,
                                min_installments = None, max_installments = None)
hist_agg_5.to_pickle(DIRECTORY + 'new_agg_5.pkl')

In [85]:
# Aggregate features, use 
# - only non-domestic transactions,
# - for first month of evaluation period.

hist_agg_6 = aggregate_features(df = hist, prefix = 'new_', suffix = '_nondom_lag1m', gb_cols = ['card_id'], agg_func = agg_func,
                                authorized = True, non_authorized = True, 
                                domestic = False, non_domestic = True,
                                max_month_lag = 1, max_week_lag = None,
                                min_installments = None, max_installments = None)
hist_agg_6.to_pickle(DIRECTORY + 'new_agg_6.pkl')

In [86]:
hist_agg = hist_agg_1.merge(hist_agg_2, how = 'left', on = 'card_id')
hist_agg = hist_agg.merge(hist_agg_3, how = 'left', on = 'card_id')
hist_agg = hist_agg.merge(hist_agg_4, how = 'left', on = 'card_id')
hist_agg = hist_agg.merge(hist_agg_5, how = 'left', on = 'card_id')
hist_agg = hist_agg.merge(hist_agg_6, how = 'left', on = 'card_id')

hist_agg.to_pickle(DIRECTORY + 'new_agg_1_6.pkl')

del hist_agg_1, hist_agg_2, hist_agg_3, hist_agg_4, hist_agg_5, hist_agg_6